# SQLAlchemy Joins

## Setup

In [1]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect

In [2]:
engine = create_engine("sqlite:///../Resources/mammal_masses.sqlite", echo=False)

In [3]:
# Reflect Database into ORM classes
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

['ea', 'na']

In [4]:
# Map Europe class
EA = Base.classes.ea

In [5]:
# Map North American class
NA = Base.classes.na

In [9]:
# create a session
session = Session(engine)

## Filtering Review

Filters are the "WHERE" clause for your select statement. 

In [22]:
tmp = session.query(NA).first()
tmp.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x108780fd0>,
 'id': 1,
 'comb_mass_g': 46082.9,
 'species': 'americana',
 'family': 'Antilocapridae',
 'sporder': 'Artiodactyla',
 'continent': None,
 'reference': '60',
 'log_mass_g': 4.66,
 'genus': 'Antilocapra',
 'status': 'extant',
 'record_id': 3898}

In [23]:
# filter North American mammals whose genus is "Antilocapra"
# query, loop over and print out animals.
mammals = session.query(NA).filter(NA.genus == 'Antilocapra').all()
for mammal in mammals:
    print(f"Family: {mammal.family}, Genus: {mammal.genus}")

Family: Antilocapridae, Genus: Antilocapra


## Joins

A SQL join combines columns from one or more tables in a relational database. 

It creates a set that can be saved as a table or used as it is. 

A JOIN is a means for combining columns from one (self-table) or more tables by using values common to each.

In [24]:
inspector = inspect(engine)
inspector.get_table_names()

['ea', 'na']

In [27]:
# Get a list of column names and types
columns = inspector.get_columns('ea')
for c in columns:
    print(c['name'], c["type"])

id INTEGER
record_id INTEGER
continent TEXT
status TEXT
sporder TEXT
family TEXT
genus TEXT
species TEXT
log_mass_g FLOAT
comb_mass_g FLOAT
reference TEXT


In [28]:
session.query(EA.sporder, NA.sporder).limit(100).all()

[('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'Artiodactyla'),
 ('Artiodactyla', 'A

In [34]:
same_sporder = session.query(EA, NA).filter(EA.sporder == NA.sporder).limit(10).all()

for record in same_sporder:
    (ea, na) = record
    print(ea.__dict__)
    print(na.__dict__)

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x10d02ed30>, 'continent': 'EA', 'id': 1, 'reference': '68', 'comb_mass_g': 37500.0, 'species': 'cervicapra', 'family': 'Bovidae', 'status': 'extant', 'record_id': 1381, 'log_mass_g': 4.57, 'genus': 'Antilope', 'sporder': 'Artiodactyla'}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x108780fd0>, 'id': 1, 'comb_mass_g': 46082.9, 'species': 'americana', 'family': 'Antilocapridae', 'sporder': 'Artiodactyla', 'continent': None, 'reference': '60', 'log_mass_g': 4.66, 'genus': 'Antilocapra', 'status': 'extant', 'record_id': 3898}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x10d02ed30>, 'continent': 'EA', 'id': 1, 'reference': '68', 'comb_mass_g': 37500.0, 'species': 'cervicapra', 'family': 'Bovidae', 'status': 'extant', 'record_id': 1381, 'log_mass_g': 4.57, 'genus': 'Antilope', 'sporder': 'Artiodactyla'}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0

In [36]:
# Return all animals from EA and NA belonging to the same sporder.
# This JOINs the data in the two tables together into a single dataset (here in the form of a tuple).
# Note: We are going to limit the results to 10 for printing
sel = [EA.family, EA.genus, EA.species, NA.family, NA.genus, NA.species]
same_sporder = session.query(*sel).filter(EA.sporder == NA.sporder).limit(10).all()

for record in same_sporder:
    (ea_fam, ea_gen, ea_spec, na_fam, na_gen, na_spec) = record
    print(
        f"The European animal '{ea_fam} {ea_gen} {ea_spec}'"
        f"belongs to the same sporder as the North American animal '{na_fam} {na_gen} {na_spec}'.")

The European animal 'Bovidae Antilope cervicapra'belongs to the same sporder as the North American animal 'Antilocapridae Antilocapra americana'.
The European animal 'Bovidae Antilope cervicapra'belongs to the same sporder as the North American animal 'Antilocapridae Capromeryx mexicana'.
The European animal 'Bovidae Antilope cervicapra'belongs to the same sporder as the North American animal 'Antilocapridae Capromeryx minor'.
The European animal 'Bovidae Antilope cervicapra'belongs to the same sporder as the North American animal 'Antilocapridae Stockoceros conklingi'.
The European animal 'Bovidae Antilope cervicapra'belongs to the same sporder as the North American animal 'Antilocapridae Stockoceros onusrosagris'.
The European animal 'Bovidae Antilope cervicapra'belongs to the same sporder as the North American animal 'Antilocapridae Tetrameryx shuleri'.
The European animal 'Bovidae Antilope cervicapra'belongs to the same sporder as the North American animal 'Bovidae Bison bison'.
Th